## Map ID to SEACell files

In [3]:
import pandas as pd
import numpy as np
import scanpy as sc
import os

dir_path = "./sc_meta_cell/cell/pan-cancer/"
output_dir = "./output_ENSG_id"

os.makedirs(output_dir, exist_ok=True)

var_name_data = pd.read_csv("./sc_meta_cell/cell/cellxgene_var_names.csv")

for file_name in os.listdir(dir_path):
    if file_name.endswith(".h5ad"):
        print(f"Processing {file_name}")
        adata = sc.read_h5ad(os.path.join(dir_path, file_name))
        sparse_matrix = adata.layers['raw']
        dense_matrix = sparse_matrix.toarray()
        sample_ids = adata.obs.index
        df = pd.DataFrame(dense_matrix)
        df.insert(0, 'sample_id', sample_ids)
        df.columns = ['sample_id'] + var_name_data['feature_id'].tolist()
        output_path = os.path.join(output_dir, file_name.replace("_SEAcells.h5ad", "_ENSG_id.csv"))
        df.to_csv(output_path, index=False)
        print(f"Saved {output_path}")

Processing blastoma_liver_partition_0_SEAcells.h5ad
Saved ./output_ENSG_id\blastoma_liver_partition_0_ENSG_id.csv
Processing blastoma_liver_partition_1_SEAcells.h5ad
Saved ./output_ENSG_id\blastoma_liver_partition_1_ENSG_id.csv


## Y Label

In [2]:
import os
import pandas as pd

# Define the folder path
folder_path = 'pan-cancer_metric'

# Lists to store results
all_cell_types = []
unique_cell_types = set()
filewise_unique_cell_types = {}

# Iterate over files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file and ends with '_purity.csv'
    if filename.endswith('_purity.csv'):
        # Get the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the file
        try:
            df = pd.read_csv(file_path)
            # Check if the 'cell_type' column exists
            if 'cell_type' in df.columns:
                cell_types = df['cell_type'].tolist()
                all_cell_types.extend(cell_types)
                unique_cell_types.update(cell_types)

                # Get unique values for each file
                filewise_unique_cell_types[filename] = set(cell_types)
            else:
                print(f"File {filename} does not contain a 'cell_type' column.")
        except Exception as e:
            print(f"Error reading file {filename}: {e}")

# Save all results to files
all_cell_types_file = 'all_cell_types.csv'
unique_cell_types_file = 'unique_cell_types.csv'
filewise_unique_cell_types_file = 'filewise_unique_cell_types.csv'

# Save all cell types
pd.DataFrame(all_cell_types, columns=['cell_type']).to_csv(all_cell_types_file, index=False)

# Save unique cell types
pd.DataFrame(list(unique_cell_types), columns=['cell_type']).to_csv(unique_cell_types_file, index=False)

# Save unique cell types for each file
filewise_data = []
for filename, unique_cells in filewise_unique_cell_types.items():
    for cell in unique_cells:
        filewise_data.append({'filename': filename, 'cell_type': cell})
filewise_df = pd.DataFrame(filewise_data)
filewise_df.to_csv(filewise_unique_cell_types_file, index=False)

# Print results with counts
print("All cell types across all files:")
print(f"Total count: {len(all_cell_types)}")
print(all_cell_types)

print("Unique cell types across all files:")
print(f"Total unique count: {len(unique_cell_types)}")
print(list(unique_cell_types))

print("Unique cell types for each file:")
for filename, unique_cells in filewise_unique_cell_types.items():
    print(f"File: {filename}")
    print(f"Unique count: {len(unique_cells)}")
    print(unique_cells)


All cell types across all files:
Total count: 14511
['myelocyte', 'classical monocyte', 'classical monocyte', 'conventional dendritic cell', 'early promyelocyte', 'classical monocyte', 'classical monocyte', 'classical monocyte', 'megakaryocyte progenitor cell', 'myelocyte', 'classical monocyte', 'conventional dendritic cell', 'hematopoietic multipotent progenitor cell', 'early promyelocyte', 'classical monocyte', 'megakaryocyte-erythroid progenitor cell', 'early promyelocyte', 'conventional dendritic cell', 'early promyelocyte', 'early promyelocyte', 'megakaryocyte progenitor cell', 'classical monocyte', 'non-classical monocyte', 'early promyelocyte', 'early promyelocyte', 'lymphoid lineage restricted progenitor cell', 'classical monocyte', 'late promyelocyte', 'hematopoietic multipotent progenitor cell', 'myelocyte', 'CD16-positive, CD56-dim natural killer cell, human', 'CD4-positive, alpha-beta memory T cell', 'early promyelocyte', 'erythroid lineage cell', 'CD16-positive, CD56-dim n

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Load the dataset
unique_cell_df = pd.read_csv('unique_cell_types.csv')

# Preprocess: lowercase and strip whitespace
unique_cell_df['cell_type_clean'] = unique_cell_df['cell_type'].str.lower().str.strip()

# Convert cell types to TF-IDF vectors
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unique_cell_df['cell_type_clean'])

# clustering with Euclidean distance
clustering_model = AgglomerativeClustering(
    n_clusters=None,  # Let the distance threshold determine clusters
    linkage='average',  # Use average linkage for clustering
    distance_threshold=1  # Adjust this value to control cluster granularity
)
labels = clustering_model.fit_predict(tfidf_matrix.toarray())

# Add the cluster labels to the dataframe
unique_cell_df['cluster'] = labels

# Group cell types by clusters
grouped_clusters = unique_cell_df.groupby('cluster')['cell_type'].apply(list).reset_index()

# Save the grouped clusters to a CSV file
grouped_clusters.to_csv('grouped_clusters.csv', index=False)

# Print the number of clusters
print(f"Total clusters: {len(grouped_clusters)}")


Total clusters: 74


#### Sort Y data

In [5]:
import os
import pandas as pd

# Define folders and files
pan_cancer_metric_folder = 'pan-cancer_metric'
y_data_folder = 'Y_data'
grouped_clusters_file = 'grouped_clusters.csv'

os.makedirs(y_data_folder, exist_ok=True)

# Load grouped clusters
print(f"Loading grouped_clusters_file: {grouped_clusters_file}")
grouped_clusters = pd.read_csv(grouped_clusters_file)

# Create a mapping from cell_type to cluster
cluster_mapping = {}
for _, row in grouped_clusters.iterrows():
    cluster = row['cluster']
    cell_types = eval(row['cell_type'])  # Convert string to list
    for cell_type in cell_types:
        cluster_mapping[cell_type] = cluster
print(f"Loaded cluster mapping: {cluster_mapping}")

# Process files in the pan-cancer_metric folder
for filename in os.listdir(pan_cancer_metric_folder):
    if filename.endswith('_purity.csv'):
        file_path = os.path.join(pan_cancer_metric_folder, filename)
        print(f"Processing file: {filename}")

        try:
            # Read the CSV file
            data = pd.read_csv(file_path)

            # Sort by the first column, extracting numerical part if applicable
            sorted_data = data.sort_values(
                by=data.columns[0],
                key=lambda x: x.str.extract(r'SEACell-(\d+)', expand=False).astype(float) 
                if x.str.contains('SEACell-').any() else x
            )

            # Map cell_type to cluster using the loaded cluster_mapping
            if 'cell_type' in sorted_data.columns:
                sorted_data['cluster'] = sorted_data['cell_type'].map(cluster_mapping)
            else:
                print(f"Warning: 'cell_type' column not found in {filename}. Skipping cluster mapping.")

            # Save to Y_data folder
            output_file_path = os.path.join(y_data_folder, filename)
            sorted_data.to_csv(output_file_path, index=False)
            print(f"Saved sorted and encoded file to {output_file_path}")

        except Exception as e:
            print(f"Error processing file {filename}: {e}")


Loading grouped_clusters_file: grouped_clusters.csv
Loaded cluster mapping: {'vein endothelial cell': 0, 'endothelial cell': 0, 'capillary endothelial cell': 0, 'non-classical monocyte': 1, 'monocyte': 1, 'classical monocyte': 1, 'paneth cell of colon': 2, 'enteroendocrine cell of colon': 2, 'plasmacytoid dendritic cell, human': 3, 'dendritic cell, human': 3, 'plasmacytoid dendritic cell': 3, 'dendritic cell': 3, 'unswitched memory B cell': 4, 'memory B cell': 4, 'CD4-positive, alpha-beta memory T cell': 5, 'activated CD4-positive, alpha-beta T cell, human': 5, 'effector memory CD8-positive, alpha-beta T cell': 5, 'central memory CD8-positive, alpha-beta T cell': 5, 'CD8-positive, alpha-beta T cell': 5, 'CD4-positive, alpha-beta cytotoxic T cell': 5, 'CD8-positive, alpha-beta memory T cell': 5, 'effector CD4-positive, alpha-beta T cell': 5, 'CD4-positive, alpha-beta T cell': 5, 'effector CD8-positive, alpha-beta T cell': 5, 'epithelial cell of alveolus of lung': 6, 'epithelial cell of 

## Construct Transcript - Protein relation

#### Add Protein as mid-node

In [6]:
import os
import pandas as pd

# Define the input and output paths
input_folder = 'output_ENSG_id'
transcript_entity_file = rf'E:\LabWork\BioMedGraphica\Entity\Transcript\biomedgraphica_transcript.csv'
protein_entity_file = rf'E:\LabWork\BioMedGraphica\Entity\Protein\biomedgraphica_protein.csv'
mapping_output_folder = 'mapping_tables'
output_folder = 'X_data'

# Ensure the output folders exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(mapping_output_folder, exist_ok=True)

# Load the transcript and protein entity data
transcript_entity_data = pd.read_csv(transcript_entity_file)
protein_entity_data = pd.read_csv(protein_entity_file)

# Iterate over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_folder, filename)
        print(f"Processing file: {filename}")

        try:
            # Read the CSV file
            data = pd.read_csv(file_path)

            # Transpose the data
            transposed_data = data.set_index(data.columns[0]).T.reset_index()
            transposed_data.rename(columns={'index': 'Ensembl_Gene_ID'}, inplace=True)

            # Merge with transcript_entity_data
            mapping_columns = ['Ensembl_Gene_ID', 'BioMedGraphica_ID']
            transcript_mapping = transcript_entity_data[mapping_columns]
            merged_data = pd.merge(
                transposed_data,
                transcript_mapping,
                on='Ensembl_Gene_ID',
                how='inner'
            )

            # Sort by BioMedGraphica_ID
            merged_data.sort_values(by='BioMedGraphica_ID', inplace=True)

            # Add BioMedGraphica_ID from protein_entity_data
            protein_ids = protein_entity_data[['BioMedGraphica_ID']].copy()
            protein_ids.loc[:, transposed_data.columns[1:]] = 0  # Fill protein sample values with 0

            # Append the protein IDs to the sorted transcript data
            final_combined_data = pd.concat(
                [merged_data, protein_ids],
                ignore_index=True,
                sort=False
            )

            # Extract the mapping table
            mapping_table = final_combined_data[['Ensembl_Gene_ID', 'BioMedGraphica_ID']].drop_duplicates()
            mapping_table.rename(columns={'Ensembl_Gene_ID': 'Original_ID'}, inplace=True)
            mapping_table.insert(0, 'Index', range(len(mapping_table)))  # Add Index as the first column

            # Save the mapping table
            mapping_table_path = os.path.join(mapping_output_folder, f"{filename.replace('.csv', '_mapping_table.csv')}")
            mapping_table.to_csv(mapping_table_path, index=False)

            # Drop the 'Ensembl_Gene_ID' column after merging
            final_combined_data.drop(columns=['Ensembl_Gene_ID'], inplace=True, errors='ignore')

            # Ensure 'BioMedGraphica_ID' is the first column
            reordered_columns = ['BioMedGraphica_ID'] + [col for col in final_combined_data.columns if col != 'BioMedGraphica_ID']
            reordered_data = final_combined_data[reordered_columns]

            # Transpose the data back
            final_data = reordered_data.set_index('BioMedGraphica_ID').T.reset_index()
            final_data.rename(columns={'index': 'Sample_ID'}, inplace=True)

            # Sort the final transposed data by the first column (Sample_ID)
            final_data.sort_values(
                by='Sample_ID', 
                key=lambda x: x.str.extract(r'SEACell-(\d+)', expand=False).astype(int), 
                inplace=True
            )

            # Save the final processed file
            output_file_path = os.path.join(output_folder, f"{filename.replace('.csv', '_processed.csv')}" )
            final_data.to_csv(output_file_path, index=False)

            print(f"Processed file saved to {output_file_path}")
            print(f"Mapping table saved to {mapping_table_path}")

        except Exception as e:
            print(f"Error processing file {filename}: {e}")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_61812\2164599508.py:17: DtypeWarning: Columns (1,2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  protein_entity_data = pd.read_csv(protein_entity_file)


Processing file: acute myeloid leukemia_bone marrow_partition_0_ENSG_id.csv
Processed file saved to X_data\acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Mapping table saved to mapping_tables\acute myeloid leukemia_bone marrow_partition_0_ENSG_id_mapping_table.csv
Processing file: acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id.csv
Processed file saved to X_data\acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Mapping table saved to mapping_tables\acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_mapping_table.csv
Processing file: adenocarcinoma_ascending colon_partition_0_ENSG_id.csv
Processed file saved to X_data\adenocarcinoma_ascending colon_partition_0_ENSG_id_processed.csv
Mapping table saved to mapping_tables\adenocarcinoma_ascending colon_partition_0_ENSG_id_mapping_table.csv
Processing file: adenocarcinoma_ileum_partition_0_ENSG_id.csv
Processed file saved to X_data\adenocarcinoma_ileum_partition_0_ENSG_id

#### Construct X_Description and Edge_index

In [7]:
import os
import pandas as pd
import json
import numpy as np

# Define file paths
mapping_folder = "mapping_tables"
protein_description_file = rf"E:\LabWork\BioMedGraphica\Entity\Protein\biomedgraphica_protein_description.csv"
transcript_description_file = rf"E:\LabWork\BioMedGraphica\Entity\Transcript\biomedgraphica_transcript_description.csv"
edge_data_file = rf"E:\LabWork\BioMedGraphica\Relation\biomedgraphica_relation.csv"

output_x_descriptions_file = "./output/X_descriptions.npy"
edge_index_output_file = "./output/edge_index.npy"

# Read the first mapping table file
mapping_files = sorted(os.listdir(mapping_folder))
if not mapping_files:
    raise FileNotFoundError(f"No files found in {mapping_folder}.")
mapping_file_path = os.path.join(mapping_folder, mapping_files[0])
entity_mapping = pd.read_csv(mapping_file_path)

# Ensure proper column names
entity_mapping.columns = ["Index", "Original_ID", "BioMedGraphica_ID"]

# Load description files
protein_description = pd.read_csv(protein_description_file)
transcript_description = pd.read_csv(transcript_description_file)

# Merge with protein description
merged_data = entity_mapping.merge(protein_description, on="BioMedGraphica_ID", how="left")

# Merge with transcript description (retain existing descriptions if present)
merged_data = merged_data.merge(transcript_description, on="BioMedGraphica_ID", how="left", suffixes=("_protein", "_transcript"))

# Combine descriptions from protein and transcript
merged_data["Description"] = merged_data["Description_protein"].fillna("") + \
                              merged_data["Description_transcript"].fillna("")

# Drop unnecessary columns
descriptions = merged_data[["Description"]]

# Convert to numpy array with shape (n, 1)
descriptions_array = descriptions.to_numpy()

# Save to .npy file
os.makedirs(os.path.dirname(output_x_descriptions_file), exist_ok=True)
np.save(output_x_descriptions_file, descriptions_array)

print(f"Descriptions saved to {output_x_descriptions_file}")


# Load edge data
edge_data_raw = pd.read_csv(edge_data_file)
edge_data = edge_data_raw[["From_ID", "To_ID", "Type"]].copy()

# Filter edge data based on entity_mapping
filtered_edge_data = edge_data[
    edge_data["From_ID"].isin(entity_mapping["BioMedGraphica_ID"]) &
    edge_data["To_ID"].isin(entity_mapping["BioMedGraphica_ID"])
]

# Map BioMedGraphica_ID to Index
id_to_index = dict(zip(entity_mapping["BioMedGraphica_ID"], entity_mapping["Index"]))
filtered_edge_data["From_Index"] = filtered_edge_data["From_ID"].map(id_to_index)
filtered_edge_data["To_Index"] = filtered_edge_data["To_ID"].map(id_to_index)

# Sort by From_Index and then by To_Index
filtered_edge_data.sort_values(by=["From_Index", "To_Index"], inplace=True)

# Construct edge_index array
edge_index = filtered_edge_data[["From_Index", "To_Index"]].dropna().astype(int).T.values

# Save edge_index to npy file
np.save(edge_index_output_file, edge_index)
print(f"Edge index saved to {edge_index_output_file} with shape {edge_index.shape}")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_61812\3936058012.py:20: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  entity_mapping = pd.read_csv(mapping_file_path)


Descriptions saved to ./output/X_descriptions.npy


C:\Users\Administrator\AppData\Local\Temp\ipykernel_61812\3936058012.py:53: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  edge_data_raw = pd.read_csv(edge_data_file)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_61812\3936058012.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edge_data["From_Index"] = filtered_edge_data["From_ID"].map(id_to_index)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_61812\3936058012.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

Edge index saved to ./output/edge_index.npy with shape (2, 33235981)


#### Construct X and Y (PyG DataBatch) 1 final batch at last

In [48]:
import os
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data, Batch

# Define folders
x_data_folder = 'X_data'
y_data_folder = 'Y_data'
edge_index_file = './output/edge_index.npy'  # Path to pre-generated edge_index file
output_file = './output/data.pt'  # Final combined Batch file

os.makedirs('./output', exist_ok=True)

# Load pre-generated edge_index (shared by all graphs)
edge_index = torch.tensor(np.load(edge_index_file), dtype=torch.long)
print(f"Loaded edge_index with shape: {edge_index.shape}")

# List to store all Data objects across all files
all_data_list = []

# Process each file in X_data
for x_filename in sorted(os.listdir(x_data_folder)):
    if x_filename.endswith('.csv'):
        x_file_path = os.path.join(x_data_folder, x_filename)
        y_file_path = os.path.join(y_data_folder, x_filename.replace('_ENSG_id_processed.csv', '_purity.csv'))

        print(f"Processing file: {x_filename}")

        # Load expression data (X) and label data (Y)
        x_data = pd.read_csv(x_file_path)  # Shape: [num_samples, num_genes]
        y_data = pd.read_csv(y_file_path)  # Shape: [num_samples, cluster/label]

        # Match samples by Sample_ID
        merged_data = pd.merge(
            x_data,
            y_data[['SEACell', 'cluster']],  # Use the cluster column for labels
            left_on='Sample_ID',
            right_on='SEACell',
            how='inner'
        )

        # Extract gene expression matrix and cluster labels
        expression_matrix = merged_data.drop(columns=['Sample_ID', 'SEACell', 'cluster']).values  # Shape: [num_samples, num_genes]
        cluster_labels = merged_data['cluster'].values  # Shape: [num_samples]

        # Create a graph for each sample in the file
        for sample_idx in range(expression_matrix.shape[0]):
            # Sample features: [num_genes]
            x = torch.tensor(expression_matrix[sample_idx], dtype=torch.float)

            # Graph label: [1] (use cluster as the label)
            y = torch.tensor([cluster_labels[sample_idx]], dtype=torch.long)

            # Create PyG Data object without edge_index
            data = Data(
                x=x.view(1, -1),  # Global features for the sample (1 row, num_genes)
                y=y  # Graph label
            )
            all_data_list.append(data)

        print(f"Processed {len(cluster_labels)} samples from {x_filename}.")

# Combine all graphs into a single large Batch
final_batch = Batch.from_data_list(all_data_list)

# Add edge_index as a global property in the Batch
final_batch.edge_index = edge_index

# Save the combined Batch to a single .pt file
torch.save(final_batch, output_file)

print(f"Saved combined Batch with {len(all_data_list)} graphs to {output_file}")


Loaded edge_index with shape: torch.Size([2, 33235981])
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_0_ENSG_id_processed.csv
Processed 250 samples from B-cell non-Hodgkin lymphoma_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_1_ENSG_id_processed.csv
Processed 48 samples from B-cell non-Hodgkin lymphoma_bone marrow_partition_1_ENSG_id_processed.csv.
Processing file: Wilms tumor_kidney_partition_0_ENSG_id_processed.csv
Processed 23 samples from Wilms tumor_kidney_partition_0_ENSG_id_processed.csv.
Processing file: acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 139 samples from acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 18 samples from acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: adenocarcinoma_

#### Construct X and Y (PyG DataBatch) 83 seperate batches at last

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data, Batch

# Define folders
x_data_folder = 'X_data'
y_data_folder = 'Y_data'
edge_index_file = './output/edge_index.npy'
final_output_file = './output/data.pt' 
batch_mapping_file = './output/batch_mapping.csv'

os.makedirs('./output', exist_ok=True)

# Load pre-generated edge_index (shared by all graphs)
edge_index = torch.tensor(np.load(edge_index_file), dtype=torch.long)
print(f"Loaded edge_index with shape: {edge_index.shape}")

# List to store batch-file mapping
batch_mapping = []

# List to store all batches
all_batches = []

# Process each file in X_data
batch_idx = 0
for x_filename in sorted(os.listdir(x_data_folder)):
    if x_filename.endswith('.csv'):
        x_file_path = os.path.join(x_data_folder, x_filename)
        y_file_path = os.path.join(y_data_folder, x_filename.replace('_ENSG_id_processed.csv', '_purity.csv'))

        print(f"Processing file: {x_filename}")

        # Load expression data (X) and label data (Y)
        x_data = pd.read_csv(x_file_path)  # Shape: [num_samples, num_genes]
        y_data = pd.read_csv(y_file_path)  # Shape: [num_samples, cluster/label]

        # Match samples by Sample_ID
        merged_data = pd.merge(
            x_data,
            y_data[['SEACell', 'cluster']],  # Use the cluster column for labels
            left_on='Sample_ID',
            right_on='SEACell',
            how='inner'
        )

        # Extract gene expression matrix and cluster labels
        expression_matrix = merged_data.drop(columns=['Sample_ID', 'SEACell', 'cluster']).values  # Shape: [num_samples, num_genes]
        cluster_labels = merged_data['cluster'].values  # Shape: [num_samples]

        # Create a graph for each sample in the file
        data_list = []
        for sample_idx in range(expression_matrix.shape[0]):
            # Sample features: [num_genes]
            x = torch.tensor(expression_matrix[sample_idx], dtype=torch.float)

            # Graph label: [1] (use cluster as the label)
            y = torch.tensor([cluster_labels[sample_idx]], dtype=torch.long)

            # Create PyG Data object
            data = Data(
                x=x.view(1, -1),  # Global features for the sample (1 row, num_genes)
                y=y  # Graph label
            )
            data_list.append(data)

        # Combine all graphs for this file into a Batch
        batch = Batch.from_data_list(data_list)
        batch.edge_index = edge_index  # Add the global edge_index

        # Append the batch to the list of all batches
        all_batches.append(batch)

        # Record batch-to-file mapping
        batch_mapping.append({
            'batch_idx': batch_idx,
            'filename': x_filename
        })

        print(f"Processed {len(data_list)} graphs in batch {batch_idx}")
        batch_idx += 1

# Save all batches as a list in a single .pt file
torch.save(all_batches, final_output_file)
print(f"Saved {len(all_batches)} batches to {final_output_file}")

# Save batch-to-file mapping to CSV
batch_mapping_df = pd.DataFrame(batch_mapping)
batch_mapping_df.to_csv(batch_mapping_file, index=False)
print(f"Saved batch-to-file mapping to {batch_mapping_file}")


Loaded edge_index with shape: torch.Size([2, 33235981])
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_0_ENSG_id_processed.csv
Processed 250 graphs in batch 0
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_1_ENSG_id_processed.csv
Processed 48 graphs in batch 1
Processing file: Wilms tumor_kidney_partition_0_ENSG_id_processed.csv
Processed 23 graphs in batch 2
Processing file: acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 139 graphs in batch 3
Processing file: acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 18 graphs in batch 4
Processing file: adenocarcinoma_ascending colon_partition_0_ENSG_id_processed.csv
Processed 21 graphs in batch 5
Processing file: adenocarcinoma_ileum_partition_0_ENSG_id_processed.csv
Processed 16 graphs in batch 6
Processing file: adenocarcinoma_rectum_partition_0_ENSG_id_processed.csv
Processed 18 graphs in batch 7
Processing file: blastoma_liver_parti

#### Construct X and Y (PyG Data) RAM usage high

In [8]:
import os
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data

# Define folders
x_data_folder = 'X_data'
y_data_folder = 'Y_data'
edge_index_file = './output/edge_index.npy'  # Path to pre-generated edge_index file
final_output_file = './output/graph.pt'  # Final global Data file

os.makedirs('./output', exist_ok=True)

# Load pre-generated edge_index (shared by all graphs)
edge_index = torch.tensor(np.load(edge_index_file), dtype=torch.long)
print(f"Loaded edge_index with shape: {edge_index.shape}")

# Initialize global storage for node features and labels
global_x = []
global_y = []

# Process each file in X_data
for x_filename in sorted(os.listdir(x_data_folder)):
    if x_filename.endswith('.csv'):
        x_file_path = os.path.join(x_data_folder, x_filename)
        y_file_path = os.path.join(y_data_folder, x_filename.replace('_ENSG_id_processed.csv', '_purity.csv'))

        print(f"Processing file: {x_filename}")

        # Load expression data (X) and label data (Y)
        x_data = pd.read_csv(x_file_path)  # Shape: [num_samples, num_genes]
        y_data = pd.read_csv(y_file_path)  # Shape: [num_samples, cluster/label]

        # Match samples by Sample_ID
        merged_data = pd.merge(
            x_data,
            y_data[['SEACell', 'cluster']],  # Use the cluster column for labels
            left_on='Sample_ID',
            right_on='SEACell',
            how='inner'
        )

        # Extract gene expression matrix and cluster labels
        expression_matrix = merged_data.drop(columns=['Sample_ID', 'SEACell', 'cluster']).values  # Shape: [num_samples, num_genes]
        cluster_labels = merged_data['cluster'].values  # Shape: [num_samples]

        # Append current file's data to global storage
        global_x.append(torch.tensor(expression_matrix, dtype=torch.float))
        global_y.append(torch.tensor(cluster_labels, dtype=torch.long))

        print(f"Processed {len(cluster_labels)} samples from {x_filename}.")

# Concatenate all data into global tensors
global_x = torch.cat(global_x, dim=0)  # Shape: [total_num_samples, num_genes]
global_y = torch.cat(global_y, dim=0)  # Shape: [total_num_samples]

# Create the global PyG Data object
global_data = Data(
    x=global_x,  # Node features
    edge_index=edge_index,  # Shared edge connections
    y=global_y  # Graph-level labels (sample labels)
)

# Save the global Data object to a .pt file
torch.save(global_data, final_output_file)
print(f"Saved global graph with {global_x.shape[0]} samples to {final_output_file}")


Loaded edge_index with shape: torch.Size([2, 33235981])
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_0_ENSG_id_processed.csv
Processed 250 samples from B-cell non-Hodgkin lymphoma_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: B-cell non-Hodgkin lymphoma_bone marrow_partition_1_ENSG_id_processed.csv
Processed 48 samples from B-cell non-Hodgkin lymphoma_bone marrow_partition_1_ENSG_id_processed.csv.
Processing file: Wilms tumor_kidney_partition_0_ENSG_id_processed.csv
Processed 23 samples from Wilms tumor_kidney_partition_0_ENSG_id_processed.csv.
Processing file: acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 139 samples from acute myeloid leukemia_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv
Processed 18 samples from acute promyelocytic leukemia_bone marrow_partition_0_ENSG_id_processed.csv.
Processing file: adenocarcinoma_

MemoryError: 

#### Sample Count

In [7]:
import os
import numpy as np
import pandas as pd

y_data_folder = 'Y_data'

total_row_count = 0

for y_filename in sorted(os.listdir(y_data_folder)):
    if y_filename.endswith('.csv'):
        y_file_path = os.path.join(y_data_folder, y_filename)
        
        print(f"Processing {y_filename}")

        y_data = pd.read_csv(y_file_path)

        row_count = len(y_data)

        print(f"Row count: {row_count}")

        total_row_count += row_count

print(f"Total row count: {total_row_count}")


Processing blastoma_liver_partition_0_purity.csv
Row count: 250
Processing blastoma_liver_partition_1_purity.csv
Row count: 37
Total row count: 287


#### Data Viewer

In [34]:
import numpy as np

edge_index = np.load("./output/edge_index.npy")
print(edge_index)
print(edge_index.shape)

X_descriptions = np.load("./output/X_descriptions.npy", allow_pickle=True)
print(X_descriptions)

[[     0      1      2 ... 451045 451045 451045]
 [246933 246934 246935 ... 354985 367459 405401]]
(2, 33235981)
[['ADP ribosylation factor 5 [Source:HGNC Symbol;Acc:HGNC:658]']
 ['mannose-6-phosphate receptor, cation dependent [Source:HGNC Symbol;Acc:HGNC:6752]']
 ['estrogen related receptor alpha [Source:HGNC Symbol;Acc:HGNC:3471]']
 ...
 ['']
 ['']
 ['']]


In [1]:
import torch

batch = torch.load('./output/data.pt')
print(batch)


[DataBatch(x=[250, 452141], y=[250], batch=[250], ptr=[251], edge_index=[2, 33235981]), DataBatch(x=[48, 452141], y=[48], batch=[48], ptr=[49], edge_index=[2, 33235981]), DataBatch(x=[23, 452141], y=[23], batch=[23], ptr=[24], edge_index=[2, 33235981]), DataBatch(x=[139, 452141], y=[139], batch=[139], ptr=[140], edge_index=[2, 33235981]), DataBatch(x=[18, 452141], y=[18], batch=[18], ptr=[19], edge_index=[2, 33235981]), DataBatch(x=[21, 452141], y=[21], batch=[21], ptr=[22], edge_index=[2, 33235981]), DataBatch(x=[16, 452141], y=[16], batch=[16], ptr=[17], edge_index=[2, 33235981]), DataBatch(x=[18, 452141], y=[18], batch=[18], ptr=[19], edge_index=[2, 33235981]), DataBatch(x=[250, 452141], y=[250], batch=[250], ptr=[251], edge_index=[2, 33235981]), DataBatch(x=[37, 452141], y=[37], batch=[37], ptr=[38], edge_index=[2, 33235981]), DataBatch(x=[12, 452141], y=[12], batch=[12], ptr=[13], edge_index=[2, 33235981]), DataBatch(x=[179, 452141], y=[179], batch=[179], ptr=[180], edge_index=[2,

In [4]:
first_batch = batch[3]

print("x:", first_batch.x)
print("y:", first_batch.y)
print("edge_index:", first_batch.edge_index)

x: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
y: tensor([ 9,  1,  1,  1, 68, 21,  5, 68,  7,  1,  1, 46, 46, 68,  1,  1, 46, 54,
        22,  5,  7, 68, 46,  4,  1,  1, 46,  1,  7, 46, 54, 22, 46,  5,  1, 46,
         5, 62, 46, 61,  9,  1, 68,  1, 21, 61,  1, 46, 46,  5,  1, 46, 48, 46,
         5, 46, 48, 68, 68,  1, 54, 46,  1,  1,  1,  5, 46,  1,  1, 54,  5,  4,
         5,  5,  1,  1,  7,  5,  9,  7,  1,  1, 69,  1,  1, 69,  9,  5,  5,  5,
        46, 46,  1,  1,  1,  1,  5, 46, 68,  4, 68, 46,  1,  1,  1, 16,  9,  1,
        68, 54,  1, 16, 46, 68, 46, 46, 16,  1,  1, 46, 22,  1, 69, 54,  9, 24,
         5, 46, 61, 24, 61,  5, 68,  1, 36,  5,  1,  4, 61])
edge_index: tensor([[     0,      1,      2,  ..., 451045, 451045, 451045],
        [246933, 246934, 246935,  ..., 3549

In [1]:
import torch
from torch_geometric.data import Batch

# Load the saved batches
batches = torch.load('./output/data.pt')

# Check the type of loaded object
if isinstance(batches, list) and all(isinstance(b, Batch) for b in batches):
    print(f"Loaded {len(batches)} individual batches. Merging into one large batch...")

    # Initialize placeholders for global properties
    global_x = []
    global_y = []
    global_batch = []  # Manually set batch indices for each sample
    global_ptr = [0]  # Start of the first graph/sample
    edge_index = None

    # Merge batches while retaining sample-level pointers
    current_sample_offset = 0
    for batch_idx, batch in enumerate(batches):
        global_x.append(batch.x)  # Append node features
        global_y.append(batch.y)  # Append graph labels

        # Manually set batch indices for each sample in this batch
        num_samples = batch.ptr.size(0) - 1  # Number of samples in this batch
        global_batch.extend([batch_idx] * num_samples)  # Assign batch index to each sample

        current_sample_offset += num_samples  # Update the sample offset
        global_ptr.append(current_sample_offset)  # Store the boundary pointer

        # Ensure all batches share the same edge_index
        if edge_index is None:
            edge_index = batch.edge_index
        elif not torch.equal(edge_index, batch.edge_index):
            raise ValueError(f"Edge indices are not consistent across batches.")

    # Concatenate all node features and labels
    global_x = torch.cat(global_x, dim=0)
    global_y = torch.cat(global_y, dim=0)

    # Create a new DataBatch with global properties
    final_batch = Batch(x=global_x, y=global_y)
    final_batch.edge_index = edge_index  # Assign the global edge_index
    final_batch.batch = torch.tensor(global_batch, dtype=torch.long)  # Custom batch indices
    final_batch.ptr = torch.tensor(global_ptr, dtype=torch.long)  # Graph/sample boundaries

    # Save the final combined DataBatch
    torch.save(final_batch, './output/final_data.pt')
    print(f"Saved final combined DataBatch with {len(global_ptr) - 1} samples to './output/final_data.pt'")
else:
    print("The loaded object is not a list of Batch objects. Please check the content of './output/data.pt'.")


Loaded 83 individual batches. Merging into one large batch...
Saved final combined DataBatch with 83 samples to './output/final_data.pt'


## View NeuroGraph Data

In [10]:
import torch

HCPAge_data = torch.load(rf'HCPAge\processed\HCPAge.pt')

print(HCPAge_data)


# 1065000 nodes, 1065 samples, 1000 nodes/sample, 1000 Regions of Interests(ROI) features/node


(Data(x=[1065000, 1000], edge_index=[2, 48551656], y=[1065]), defaultdict(<class 'dict'>, {'x': tensor([      0,    1000,    2000,  ..., 1063000, 1064000, 1065000]), 'edge_index': tensor([       0,    47450,    92144,  ..., 48457754, 48505876, 48551656]), 'y': tensor([   0,    1,    2,  ..., 1063, 1064, 1065])}))


In [14]:
import torch

# Load the processed HCPAge dataset
data, meta = torch.load(r'HCPAge/processed/HCPAge.pt')

print("Data:", data)
print("Meta:", meta)


# Extract the node features (x) and node offsets
x = data.x  # Node features
x_offsets = meta['x']  # Node offsets

print("Node features (x):", x.shape)  # torch.Size([1065000, 1000])
print("Node offsets:", x_offsets)  # [      0,    1000,    2000,  ..., 1063000, 1064000, 1065000]

i = 1

# Extract the node features for the i-th graph
start_idx = x_offsets[i]
end_idx = x_offsets[i + 1]

x_i = x[start_idx:end_idx]

print(f"Graph {i}: Node features shape = {x_i.shape}")
print(f"Graph {i}: Node features =\n{x_i}")


Data: Data(x=[1065000, 1000], edge_index=[2, 48551656], y=[1065])
Meta: defaultdict(<class 'dict'>, {'x': tensor([      0,    1000,    2000,  ..., 1063000, 1064000, 1065000]), 'edge_index': tensor([       0,    47450,    92144,  ..., 48457754, 48505876, 48551656]), 'y': tensor([   0,    1,    2,  ..., 1063, 1064, 1065])})
Node features (x): torch.Size([1065000, 1000])
Node offsets: tensor([      0,    1000,    2000,  ..., 1063000, 1064000, 1065000])
Graph 1: Node features shape = torch.Size([1000, 1000])
Graph 1: Node features =
tensor([[0.0000, 0.3982, 0.3452,  ..., 0.3701, 0.1743, 0.1266],
        [0.3982, 0.0000, 0.1934,  ..., 0.2365, 0.0539, 0.1022],
        [0.3452, 0.1934, 0.0000,  ..., 0.2629, 0.2640, 0.2520],
        ...,
        [0.3701, 0.2365, 0.2629,  ..., 0.0000, 0.4140, 0.2668],
        [0.1743, 0.0539, 0.2640,  ..., 0.4140, 0.0000, 0.4163],
        [0.1266, 0.1022, 0.2520,  ..., 0.2668, 0.4163, 0.0000]])


In [34]:
y = data.y
print("Graph labels (y):", y)
print("Shape of y:", y.shape)

i = 124
label_i = y[i]
print(f"Graph {i} label:", label_i.item())

Graph labels (y): tensor([1, 1, 2,  ..., 1, 1, 1])
Shape of y: torch.Size([1065])
Graph 124 label: 1


In [35]:
data, meta = torch.load(r'HCPAge/processed/HCPAge.pt')

edge_index_offsets = meta['edge_index']

# edge numbers for each graph
num_edges_per_graph = edge_index_offsets[1:] - edge_index_offsets[:-1]

print("Number of edges per graph:", num_edges_per_graph)
print("First 5 graphs' edges:", num_edges_per_graph[:5])


Number of edges per graph: tensor([47450, 44694, 48216,  ..., 45572, 48122, 45780])
First 5 graphs' edges: tensor([47450, 44694, 48216, 47108, 39448])
